# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 12:51PM,238663,20,7652907-BO,"ACI Healthcare USA, Inc.",Released
1,Jun 2 2022 12:51PM,238663,20,7713510-BO,"ACI Healthcare USA, Inc.",Released
2,Jun 2 2022 12:51PM,238663,20,7749038-BO,"ACI Healthcare USA, Inc.",Released
3,Jun 2 2022 12:51PM,238663,20,7760027-BO,"ACI Healthcare USA, Inc.",Released
4,Jun 2 2022 12:51PM,238663,20,7708597-BO,"ACI Healthcare USA, Inc.",Released
5,Jun 2 2022 12:51PM,238663,20,7708599-BO,"ACI Healthcare USA, Inc.",Released
6,Jun 2 2022 12:51PM,238663,20,7708602-BO,"ACI Healthcare USA, Inc.",Released
7,Jun 2 2022 12:51PM,238663,20,7778537-BO,"ACI Healthcare USA, Inc.",Released
8,Jun 2 2022 12:51PM,238663,20,7790363-BO,"ACI Healthcare USA, Inc.",Released
9,Jun 2 2022 12:51PM,238663,20,7741164-BO1,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,238659,Released,1
43,238660,Released,1
44,238661,Released,8
45,238662,Released,2
46,238663,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238659,NaN,NaN,1.0
238660,NaN,NaN,1.0
238661,NaN,NaN,8.0
238662,NaN,NaN,2.0
238663,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238553,0.0,1.0,0.0
238556,0.0,0.0,2.0
238557,0.0,0.0,5.0
238564,0.0,6.0,1.0
238568,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238553,0,1,0
238556,0,0,2
238557,0,0,5
238564,0,6,1
238568,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238553,0,1,0
1,238556,0,0,2
2,238557,0,0,5
3,238564,0,6,1
4,238568,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238553,,1,
1,238556,,,2
2,238557,,,5
3,238564,,6,1
4,238568,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc."
11,Jun 2 2022 12:49PM,238659,10,Bio-PRF
12,Jun 2 2022 12:48PM,238661,12,Hush Hush
20,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC"
22,Jun 2 2022 12:45PM,238660,10,Bio-PRF
23,Jun 2 2022 12:31PM,238656,10,Aerogen LTD.
26,Jun 2 2022 12:24PM,238657,19,"AdvaGen Pharma, Ltd"
28,Jun 2 2022 12:02PM,238654,16,Seqirus
29,Jun 2 2022 11:48AM,238653,12,Hush Hush
30,Jun 2 2022 11:24AM,238650,19,VITABIOTICS USA INC.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc.",,,11
1,Jun 2 2022 12:49PM,238659,10,Bio-PRF,,,1
2,Jun 2 2022 12:48PM,238661,12,Hush Hush,,,8
3,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC",,,2
4,Jun 2 2022 12:45PM,238660,10,Bio-PRF,,,1
5,Jun 2 2022 12:31PM,238656,10,Aerogen LTD.,,,3
6,Jun 2 2022 12:24PM,238657,19,"AdvaGen Pharma, Ltd",,,2
7,Jun 2 2022 12:02PM,238654,16,Seqirus,,,1
8,Jun 2 2022 11:48AM,238653,12,Hush Hush,,1,
9,Jun 2 2022 11:24AM,238650,19,VITABIOTICS USA INC.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc.",11,,
1,Jun 2 2022 12:49PM,238659,10,Bio-PRF,1,,
2,Jun 2 2022 12:48PM,238661,12,Hush Hush,8,,
3,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC",2,,
4,Jun 2 2022 12:45PM,238660,10,Bio-PRF,1,,
5,Jun 2 2022 12:31PM,238656,10,Aerogen LTD.,3,,
6,Jun 2 2022 12:24PM,238657,19,"AdvaGen Pharma, Ltd",2,,
7,Jun 2 2022 12:02PM,238654,16,Seqirus,1,,
8,Jun 2 2022 11:48AM,238653,12,Hush Hush,,1,
9,Jun 2 2022 11:24AM,238650,19,VITABIOTICS USA INC.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc.",11,,
1,Jun 2 2022 12:49PM,238659,10,Bio-PRF,1,,
2,Jun 2 2022 12:48PM,238661,12,Hush Hush,8,,
3,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC",2,,
4,Jun 2 2022 12:45PM,238660,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc.",11.0,NaN,NaN
1,Jun 2 2022 12:49PM,238659,10,Bio-PRF,1.0,NaN,NaN
2,Jun 2 2022 12:48PM,238661,12,Hush Hush,8.0,NaN,NaN
3,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
4,Jun 2 2022 12:45PM,238660,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc.",11.0,0.0,0.0
1,Jun 2 2022 12:49PM,238659,10,Bio-PRF,1.0,0.0,0.0
2,Jun 2 2022 12:48PM,238661,12,Hush Hush,8.0,0.0,0.0
3,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
4,Jun 2 2022 12:45PM,238660,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2386378,59.0,2.0,4.0
12,1431674,16.0,4.0,0.0
15,1431714,17.0,37.0,15.0
16,954490,2.0,2.0,0.0
19,954555,13.0,24.0,31.0
20,1431610,14.0,8.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2386378,59.0,2.0,4.0
1,12,1431674,16.0,4.0,0.0
2,15,1431714,17.0,37.0,15.0
3,16,954490,2.0,2.0,0.0
4,19,954555,13.0,24.0,31.0
5,20,1431610,14.0,8.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,59.0,2.0,4.0
1,12,16.0,4.0,0.0
2,15,17.0,37.0,15.0
3,16,2.0,2.0,0.0
4,19,13.0,24.0,31.0
5,20,14.0,8.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,59.0
1,12,Released,16.0
2,15,Released,17.0
3,16,Released,2.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,4.0,0.0,15.0,0.0,31.0,0.0
Executing,2.0,4.0,37.0,2.0,24.0,8.0
Released,59.0,16.0,17.0,2.0,13.0,14.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,4.0,0.0,15.0,0.0,31.0,0.0
1,Executing,2.0,4.0,37.0,2.0,24.0,8.0
2,Released,59.0,16.0,17.0,2.0,13.0,14.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,4.0,0.0,15.0,0.0,31.0,0.0
1,Executing,2.0,4.0,37.0,2.0,24.0,8.0
2,Released,59.0,16.0,17.0,2.0,13.0,14.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()